In [1]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

In [3]:
from dash import Dash, html, Output, Input
from jupyter_dash import JupyterDash
import dash_leaflet as dl
import dash_leaflet.express as dlx
from dash_extensions.enrich import html, DashProxy
from dash_extensions.javascript import arrow_function, assign
# standalone map
import folium
from folium import plugins
# data manipulation
import numpy as np
import pandas as pd
import json
import geojson
import geopandas as gpd

data_directory = '/Users/felixschulz/Library/CloudStorage/OneDrive-Persönlich/Dokumente/Uni/Data Science Project 2022/'

In [4]:
# load germany maps & polygons
#bund = gpd.read_file(data_directory + 'Bundeskarte.geojson').to_crs(4326)
land = gpd.read_file(data_directory + 'data/gadm41_DEU_shp/gadm41_DEU_1.shp').to_crs(4326)
kreis = gpd.read_file(data_directory + "data/gadm41_DEU_shp/gadm41_DEU_2.shp").to_crs(4326)
polygon = gpd.read_file(data_directory + "modules/dashboard/data/BB_final.json").set_crs(25833, allow_override = True).to_crs(4326)

In [7]:
land.loc[land.NAME_1 == "Brandenburg"].centroid

/var/folders/56/097r5m_53z97wd0wwmqws76m0000gn/T/ipykernel_42992/4103741669.py:1: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




3    POINT (13.39777 52.47288)
dtype: geometry

In [8]:
polygon_offset = polygon.copy().to_crs(3043)
polygon_offset['geometry'] = polygon_offset.geometry.buffer(-12, single_sided = True)
polygon_offset['area'] = polygon_offset['geometry'].area / 10**3
polygon_offset['geometry'] = polygon_offset.geometry.to_crs(4326)

In [9]:
# add random score between 0 and 10 to kreis
np.random.seed(123)
kreis.insert(
    13,
    "Score",
    np.random.uniform(0, 11, size=len(kreis))
)

In [13]:
kreis.to_file(data_directory + "dashboard/assets/kreis.json", driver="GeoJSON")

In [6]:
polygon.to_file(data_directory + "modules/dashboard/assets/polygon.json", driver="GeoJSON")

In [6]:
'''
with open("kreis.json", "w") as f:
    f.write(kreis.to_json())
with open("land.json", "w") as f:
    f.write(land.to_json())
with open("polygone.json", "w") as f:
    f.write(polygon_offset.to_json())
'''

'\nwith open("kreis.json", "w") as f:\n    f.write(kreis.to_json())\nwith open("land.json", "w") as f:\n    f.write(land.to_json())\nwith open("polygone.json", "w") as f:\n    f.write(polygon_offset.to_json())\n'

In [11]:
# feed kreis in dash app
kreis_path = data_directory + "dashboard/assets/kreis.json"
# load kreis
with open(kreis_path, 'r') as f:
    kreis = [f['properties'] for f in json.loads(f.read())['features']]

# feed land in dash app
land_path = "assets/land.json"
# load kreis
with open(land_path, 'r') as f:
    land = [f['properties'] for f in json.loads(f.read())['features']]

# feed polygone in dash app
polygon_path = "assets/polygone.json"
# load kreis
with open(polygon_path, 'r') as f:
    polygone = [f['properties'] for f in json.loads(f.read())['features']]

FileNotFoundError: [Errno 2] No such file or directory: '/Users/felixschulz/Library/CloudStorage/OneDrive-Persönlich/Dokumente/Uni/Data Science Project 2022/dashboard/assets/kreis.json'

In [8]:
# define info box
def get_info(feature=None):
    header = [html.H4("Fitness für Ohren")]
    if not feature:
        return header + [html.P("Über Provinz hovern")]
    return header + [html.B(feature["properties"]["NAME_3"]), html.Br(),
                     "{:.2f} Lustnauer Score".format(feature["properties"]["Score"])]

classes = [0, 2, 4, 6, 8, 10]
colorscale = ['#ffd34c', '#e5f3e5', '#99d099', '#66b967', '#32a234', '#008b02']
style = dict(weight=2, opacity=1, color='white', dashArray='3', fillOpacity=0.7)
# Create colorbar.
ctg = ["{}+".format(cls, classes[i + 1]) for i, cls in enumerate(classes[:-1])]
colorbar = dlx.categorical_colorbar(categories=ctg, colorscale=colorscale, width=300, height=30, position="bottomleft")
# Geojson rendering logic, must be JavaScript as it is executed in clientside.
style_handle = assign("""function(feature, context){
    const {classes, colorscale, style, colorProp} = context.props.hideout;  // get props from hideout
    const value = feature.properties[colorProp];  // get value the determines the color
    for (let i = 0; i < classes.length; ++i) {
        if (value > classes[i]) {
            style.fillColor = colorscale[i];  // set the fill color according to the class
        }
    }
    return style;
}""")
# Create geojsons.
kreis_geojson = dl.GeoJSON(url=f"/{kreis_path}",  # url to geojson file
                     options=dict(style=style_handle),  # how to style each polygon
                     zoomToBounds=True,  # when true, zooms to bounds when data changes (e.g. on load)
                     zoomToBoundsOnClick=True,  # when true, zooms to bounds of feature (e.g. polygon) on click
                     hoverStyle=arrow_function(dict(weight=5, color='#666', dashArray='')),  # style applied on hover
                     hideout=dict(colorscale=colorscale, classes=classes, style=style, colorProp="Score"),
                     id="geojson")
polygon_geojson = dl.GeoJSON(url=f"/{polygon_path}",  # url to geojson file
                     #options=dict(style=style_handle),  # how to style each polygon
                     #zoomToBounds=True,  # when true, zooms to bounds when data changes (e.g. on load)
                     zoomToBoundsOnClick=True,  # when true, zooms to bounds of feature (e.g. polygon) on click
                     #hoverStyle=arrow_function(dict(weight=5, color='#666', dashArray='')),  # style applied on hover
                     #hideout=dict(colorscale=colorscale, classes=classes, style=style, colorProp="Score"),
                     id="geojson")


# Create info control.
info = html.Div(children=get_info(), id="info", className="info",
                style={"position": "absolute", "top": "10px", "right": "50px", "z-index": "1000"})
# app backend
app = Dash(prevent_initial_callbacks=True)
'''
app.layout = html.Div(
    # show map
    [dl.Map(
        children=[#dl.LayersControl(
                  #    dl.BaseLayer(
                          dl.TileLayer(),
                  #        ),
                      #dl.Overlay(dl.LayerGroup(polygon_geojson), name = 'Ohren', checked = True)
                  #  ),
                  dl.GestureHandling(), kreis_geojson, colorbar, info
                  ],
    # set max bounds and zoom
    maxBounds = [
        # north-east
        [55.5, 15.7],
        # south-west
        [47.1, 5.4]
        ],
    zoom = 6,
    minZoom = 6,
    # set style in dash board                  
    style={'width': '50%', 'height': '75vh', 'margin': "auto", "display": "block"})],
    id="map")
'''
app.layout = html.Div([dl.Map(children=[dl.TileLayer(), kreis_geojson, colorbar, info])],
                      style={'width': '50%', 'height': '75vh', 'margin': "auto", "display": "block"}, id="map")

@app.callback(
    Output("info", "children"),
    [Input("geojson", "hover_feature")]
)

def info_hover(feature):
    return get_info(feature)

In [ ]:

# preparing a color scale and -bar
classes = [0, 2, 4, 6, 8, 10]
colorscale = ['#fde725', '#5ec962', '#21918c', '#3b528b', '#440154']
style = dict(weight=2, opacity=1, color='white', dashArray='3', fillOpacity=0.7)
# Create colorbar.
ctg = ["{}+".format(cls, classes[i + 1]) for i, cls in enumerate(classes[:-1])]
colorbar = dlx.categorical_colorbar(categories=ctg, colorscale=colorscale, width=300, height=30, position="bottomleft")
# Geojson rendering logic, must be JavaScript as it is executed in clientside.
style_handle = assign("""function(feature, context){
    const {classes, colorscale, style, colorProp} = context.props.hideout;  // get props from hideout
    const value = feature.properties["link_id"];  // get value the determines the color
    if (value === null) {
        style.fillColor = classes[classes.length - 1];
        return style;
        }
    for (let i = 0; i < classes.length; ++i) {
        if (value > classes[i]) {
            style.fillColor = colorscale[i];  // set the fill color according to the class
        }
    }
    if (style.fillColor === undefined) {
        style.fillColor = classes[classes.length - 1];
        }
    return style;
}""")

---

In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [7]:
data_directory = '/Users/felixschulz/Library/CloudStorage/OneDrive-Persönlich/Dokumente/Uni/Data Science Project 2022/'

In [5]:
model = {
    "irradiation": 0.2,
    "distance": 0.25,
    "terrain": .05  
}

In [11]:
# process polygons
polygon = gpd.read_file(data_directory + "modules/dashboard/data/BB_final.json")
# apply buffer to polygons
polygon['geometry'] = polygon.geometry.apply(lambda x: x.buffer(-12) if x is not None else None)
# get the overall area
polygon['suitable_area'] = polygon['geometry'].area
# get the irradiation score
polygon["irradiation_score"] = polygon.irradiation.apply(lambda x: (polygon.irradiation.max() - x) / (polygon.irradiation.max() - polygon.irradiation.min()))
# get the distance score
polygon["distance_score"] = polygon["distance"].apply(lambda x: (polygon["distance"].max() - x) / (polygon["distance"].max() - polygon["distance"].min()))
# get the overall score
polygon["overall_score"] = (model["irradiation"] * polygon["irradiation_score"] + 
                    model["distance"] * polygon["distance_score"] +
                    model["terrain"] * polygon["terrain_suitability"])
# write and deploy the result to a json file
polygon.to_crs(4326).to_file(data_directory + "modules/dashboard/assets/polygon.json", driver="GeoJSON")

In [8]:
# read the kreis data
kreis = gpd.read_file(data_directory + "data/gadm41_DEU_shp/gadm41_DEU_2.shp").to_crs(25833)
# filter for brandenburg
kreis = kreis[kreis['NAME_1'] == 'Brandenburg']
# get mean statistics
tmp = gpd.sjoin(kreis, polygon, how = 'inner', predicate = 'intersects').reset_index()
tmp["link_id_individual"] = tmp.link_id + "_" + str(tmp.id)
# Define a lambda function to compute the weighted mean:
wm = lambda x: np.average(x[~x.isna()], weights = tmp.loc[x[~x.isna()].index, "suitable_area"])
#
kreis_stats = tmp.groupby('NAME_2').agg({'link_id_individual': "count",
                                         'terrain_suitability': wm, 
                                         'irradiation_score': wm, 
                                         'distance_score': wm,
                                         'overall_score': wm}).reset_index()
pd.merge(kreis, kreis_stats, left_on = 'NAME_2', right_on = 'NAME_2', how = 'left').to_crs(4326)#.to_file(data_directory + "modules/dashboard/assets/kreis.json", driver = 'GeoJSON')

NameError: name 'polygon' is not defined

In [18]:
gemeinde

,GID_3,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,NL_NAME_2,NAME_3,VARNAME_3,NL_NAME_3,TYPE_3,ENGTYPE_3,CC_3,HASC_3,geometry
2029,DEU.4.1.1_1,DEU,Germany,DEU.4_1,Brandenburg,NA,DEU.4.1_1,Barnim,NA,Ahrensfelde,NA,NA,Amtsfreie Gemeinde,Municipality,120600005,NA,"POLYGON ((405795.741 5822479.062, 405596.938 5..."
2030,DEU.4.1.2_1,DEU,Germany,DEU.4_1,Brandenburg,NA,DEU.4.1_1,Barnim,NA,Bernau bei Berlin,NA,NA,Amtsfreie Gemeinde,Municipality,120600020,NA,"POLYGON ((407803.875 5831724.239, 407691.226 5..."
2031,DEU.4.1.3_1,DEU,Germany,DEU.4_1,Brandenburg,NA,DEU.4.1_1,Barnim,NA,Biesenthal-Barnim,NA,NA,Amt,Municipality,120605003,NA,"POLYGON ((410429.753 5849019.893, 412294.281 5..."
2032,DEU.4.1.4_1,DEU,Germany,DEU.4_1,Brandenburg,NA,DEU.4.1_1,Barnim,NA,Britz-Chorin-Oderberg,NA,NA,Amt,Municipality,120605011,NA,"POLYGON ((423422.226 5849843.202, 423459.264 5..."
2033,DEU.4.1.5_1,DEU,Germany,DEU.4_1,Brandenburg,NA,DEU.4.1_1,Barnim,NA,Eberswalde,NA,NA,Amtsfreie Gemeinde,Municipality,120600052,NA,"POLYGON ((410411.628 5849700.201, 410998.173 5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2224,DEU.4.18.9_1,DEU,Germany,DEU.4_1,Brandenburg,NA,DEU.4.18_1,Uckermark,NA,Oder-Welse,NA,NA,Amt,Municipality,120735310,NA,"POLYGON ((442407.581 5868476.678, 442444.922 5..."
2225,DEU.4.18.10_1,DEU,Germany,DEU.4_1,Brandenburg,NA,DEU.4.18_1,Uckermark,NA,Prenzlau,NA,NA,Amtsfreie Gemeinde,Municipality,120730452,NA,"POLYGON ((424424.039 5916569.827, 424521.511 5..."
2226,DEU.4.18.11_1,DEU,Germany,DEU.4_1,Brandenburg,NA,DEU.4.18_1,Uckermark,NA,Schwedt/Oder,NA,NA,Amtsfreie Gemeinde,Municipality,120730532,NA,"POLYGON ((446767.308 5891768.951, 447167.456 5..."
2227,DEU.4.18.12_1,DEU,Germany,DEU.4_1,Brandenburg,NA,DEU.4.18_1,Uckermark,NA,Templin,NA,NA,Amtsfreie Gemeinde,Municipality,120730572,NA,"POLYGON ((410885.184 5873209.513, 409617.686 5..."


In [17]:
# read the kreis data
gemeinde = gpd.read_file(data_directory + "data/gadm41_DEU_shp/gadm41_DEU_3.shp").to_crs(25833)
# filter for brandenburg
gemeinde = gemeinde[gemeinde['NAME_1'] == 'Brandenburg']
# get mean statistics
tmp = gpd.sjoin(gemeinde, polygon, how = 'inner', predicate = 'intersects').reset_index()
tmp["link_id_individual"] = tmp.link_id + "_" + str(tmp.id)
# Define a lambda function to compute the weighted mean:
wm = lambda x: np.average(x[~x.isna()], weights = tmp.loc[x[~x.isna()].index, "suitable_area"]) if len(x[~x.isna()]) != 0 else None
gemeinde_stats = tmp.groupby('NAME_3').agg({'suitable_area': "sum",
                                            'link_id_individual': "count",
                                         'terrain_suitability': wm, 
                                         'irradiation_score': wm, 
                                         'distance_score': wm,
                                         'overall_score': wm}).reset_index()
pd.merge(gemeinde, gemeinde_stats, left_on = 'NAME_3', right_on = 'NAME_3', how = 'left').to_crs(4326).to_file(data_directory + "modules/dashboard/assets/gemeinde.json", driver = 'GeoJSON')

NameError: name 'polygon' is not defined

---

In [1]:
from dash import Dash, html, Output, Input, dcc
from dash.exceptions import PreventUpdate
from jupyter_dash import JupyterDash
import dash_leaflet as dl
import dash_leaflet.express as dlx
import dash_bootstrap_components as dbc
from dash_extensions.enrich import html, DashProxy
from dash_extensions.javascript import arrow_function, assign
import json
import plotly.express as px

import base64
###
# Driveways
###

polygon_geojson = dl.GeoJSON(url=f"assets/BB_polygons_final.json", 
                             id="polygon_geojson",
                             options = {"style": {"color": "#3D426B", 
                                                  "opacity": 0.8, 
                                                  "fillOpacity": 0.2, 
                                                  "weight": 2}},
                             hoverStyle = arrow_function({"color": "#3D426B", 
                                                          "fillColor":"#779ecb", 
                                                          "fillOpacity": 0.2, 
                                                          "opacity": 1, 
                                                          "weight": 2}))

###
# Gemeinde
###

gemeinde_geojson = dl.GeoJSON(url=f"/assets/BB_gemeinde_final.json",  # url to geojson file
                     #zoomToBounds=True,  # when true, zooms to bounds when data changes (e.g. on load)
                     zoomToBoundsOnClick=True,  # when true, zooms to bounds of feature (e.g. polygon) on click
                     #hoverStyle=arrow_function(dict(weight=5, color='#666', dashArray='')),  # style applied on hover
                     #hideout = dict(colorscale=colorscale, classes=classes, style=style, colorProp="Score"),
                     #options = dict(style=style_handle),
                     options={"style":{"color":"grey", "opacity": 0.5, "fillOpacity": 0.2, "weight": 1}},
                     hoverStyle = arrow_function({"color":"purple", "fillColor":"grey", "fillOpacity": 0.2, "opacity": 0.8, "weight": 2}),
                     id="gemeinde_geojson")

###
# Kreis
###

kreis_geojson = dl.GeoJSON(url=f"/assets/BB_kreis_final.json",  # url to geojson file
                     #zoomToBounds=True,  # when true, zooms to bounds when data changes (e.g. on load)
                     zoomToBoundsOnClick=True,  # when true, zooms to bounds of feature (e.g. polygon) on click
                     options={"style":{"color":"grey", "opacity": 0.5, "fillOpacity": 0.2, "weight": 1}},
                     hoverStyle=arrow_function({"color":"purple", "fillColor":"grey", "fillOpacity": 0.2, "opacity": 0.8, "weight": 2}),
                     id="kreis_geojson")

# define info box
def get_info(feature=None):
    header = [html.H4("Gemeindestatistiken")]
    if not feature:
        return header + [html.P("Zoome für weitere Statistiken\nauf eine Gemeinde")]
    if feature["properties"]["NAME_3"] is not None:
        if feature["properties"]["suitable_area"] is not None:
            return header + [html.B(feature["properties"]["NAME_3"]), html.Br(),
                            "Potentialfläche in m2: {:.2f}".format(feature["properties"]["suitable_area"]), html.Br(),
                            "Durchschnittliche Wertung: {:.2f}".format(feature["properties"]["overall_score"]), html.Br(),
                            "Klicke für weitere Statistiken auf eine Potentialfläche!"]
        else:
            return header + [html.B(feature["properties"]["NAME_3"]), html.Br(),
                            "Potentialfläche in m2: 0", html.Br(),
                            "Durchschnittliche Wertung: --", html.Br(),
                            "Klicke für weitere Statistiken auf eine Potentialfläche!"]
info = html.Div(children=get_info(), id="info", className="info",
                style={"position": "absolute", "top": "10px", "right": "50px", "z-index": "1000"})


tile_layer = dl.TileLayer(url="https://tiles.stadiamaps.com/tiles/alidade_smooth/{z}/{x}/{y}{r}.png", id="tile_layer", 
                          attribution = '&copy; <a href="https://stadiamaps.com/">Stadia Maps</a>, &copy; <a href="https://openmaptiles.org/">OpenMapTiles</a> &copy; <a href="http://openstreetmap.org">OpenStreetMap</a> contributors')

###
# App assembly
###

# core
app = JupyterDash(prevent_initial_callbacks = True, 
                  external_stylesheets = [dbc.themes.BOOTSTRAP])
# layout
app.layout = html.Div([
    dbc.Row([
        dbc.Col([dl.Map(children = [tile_layer, 
                                          dl.Pane([kreis_geojson], style = {"zIndex": 200}), 
                                          dl.Pane([polygon_geojson], style = {"zIndex": 300}), 
                                          info], 
                        style={'width': '100%', 'height': '75vh', 'margin': "auto", "display": "block"},
                        id = "map", center = [52.47288, 13.39777], zoom = 7)],
                width = {"size": 12, "offset": 0})
        ]),
    html.Div(id = "info-panel")
    
])

###
# Interactivity
###

# a lagged state variable
state = {"zoom": 7, "clickedPolygon": ""}

# a callback that controls what polygons are shown
@app.callback(Output("map", "children"), Input("map", "zoom"))
def func(viewport):
    #if ((viewport > 12) & (state["zoom"] <= 12)):
    #    state["zoom"] = viewport
    #    print("A2")
    #    return [tile_layer, polygon_geojson, info]
    if ((viewport > 8) & (state["zoom"] <= 8)):#(((viewport > 7) & (viewport <= 12)) & ((state["zoom"] <= 7) | (state["zoom"] > 12))):
        state["zoom"] = viewport
        return [tile_layer, 
                dl.Pane([gemeinde_geojson], style = {"zIndex": 250}), 
                dl.Pane([polygon_geojson], style = {"zIndex": 300}), 
                info]
    elif ((viewport <= 8) & (state["zoom"] > 8)):
        state["zoom"] = viewport
        return [tile_layer, 
                dl.Pane([kreis_geojson], style = {"zIndex": 200}), 
                dl.Pane([polygon_geojson], style = {"zIndex": 300}), 
                info]
    else:
        state["zoom"] = viewport
        raise PreventUpdate()

# a callback that controls the info box
@app.callback(Output("info", "children"), [Input("gemeinde_geojson", "hover_feature")])
def info_hover(feature):
    return get_info(feature)


# a callback that updates the info panel
@app.callback(Output("info-panel", "children"), Input("polygon_geojson", "click_feature"))
def info_click(feature):
    if feature is not None:
        # try and get core data
        link_id = feature["properties"]["link_id"] + \
            ", " + str(int(feature["properties"]["id"]))
        if feature["properties"]["name"] is not None:
            link_name = f" (" + feature["properties"]["name"] + ")"
        else:
            link_name = ""
        suitable_area = "{:.2f}".format(feature["properties"]["suitable_area"])
        overall_score = "{:.2f}".format(feature["properties"]["overall_score"])
        # make pie chart
        df = px.data.tips()
        fig = px.pie(df, values='tip', names='day')
        fig.layout.update()
        fig.update_traces(textposition='inside')
        fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide',
                          margin = dict(l=0, r=0, t=0, b=0), showlegend = False)
        # get image of terrain
        try:
            with open("assets/imagery/" + feature["properties"]["link_id"] + "_" + str(int(feature["properties"]["id"])) + ".png", "rb") as image_file:
                terrain_image = "data:image/png;base64,{}".format(
                    base64.b64encode(image_file.read()).decode('ascii'))
        except:
            terrain_image = None
        # terrain data
        terrain_roughness = "{:.2f}".format(
            feature["properties"]["terrain_roughness"])
        terrain_high = "{:.2f}".format(feature["properties"]["terrain_high"])
        terrain_low = "{:.2f}".format(feature["properties"]["terrain_low"])
        terrain_score = "{:.2f}".format(feature["properties"]["terrain_score"])
        # get distance data
        distance = "{:.2f}".format(feature["properties"]["distance"])
        distance_score = "{:.2f}".format(
            feature["properties"]["distance_score"])
        # get irradiation data
        irradiation = "{:.2f}".format(feature["properties"]["irradiation"])
        irradiation_score = "{:.2f}".format(
            feature["properties"]["irradiation_score"])
        #
        return [dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4("Fläche", className="card-title"),
                        html.B(link_id + link_name), html.Br(),
                        html.Span("Potentialfläche in m2: ", style={
                                  "color": "grey"}), suitable_area, html.Br(),
                        html.Span("Wertung: ", style={
                                  "color": "grey"}), overall_score, html.Br(),
                        html.Span("Rang: ", style={"color": "grey"}), "--"])
                ], class_name="h-100")
            ], width={"size": 4}),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4("Landbedeckung", className="card-title"),
                        dbc.Row([
                            dbc.Col([html.Img(src=terrain_image, width="100px", height="100px")], width={
                                    "size": 4}),
                            dbc.Col([html.Span("Nutzbare Fläche in m2: ", style={"color": "grey"}), "--", html.Br(),
                                     html.Span("Wertung: ", style={
                                               "color": "grey"}), "--", html.Br(),
                                     html.Span("Rang: ", style={"color": "grey"}), "--"], width={"size": 4}),
                            dbc.Col([dcc.Graph(id="graph", figure=fig, style={
                                    'width': '150px', 'height': '150px'})], width={"size": 4})
                        ])
                    ]),
                ], class_name="h-100")
            ], width={"size": 8})], class_name="m-1"),
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H4("Geländebeschaffenheit",
                                    className="card-title"),
                            dbc.Row([
                                dbc.Col([html.Img(src=terrain_image, width="100px", height="100px")], width={
                                        "size": 6}),
                                dbc.Col([html.Span("Mittlere Abweichung der Steigung: ", style={"color": "grey"}), terrain_roughness, html.Br(),
                                         html.Span("Hochpunkt: ", style={"color": "grey"}), terrain_high, html.Span(
                                             "  Tiefpunkt: ", style={"color": "grey"}), terrain_low, html.Br(),
                                         html.Span("Wertung: ", style={
                                                   "color": "grey"}), terrain_score, html.Br(),
                                         html.Span("Rang: ", style={"color": "grey"}), "--"], width={"size": 6})
                            ])
                        ]),
                    ], class_name="h-100")
                ], width={"size": 4}),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H4("Netzanschluss", className="card-title"),
                            html.Span("Distanz zum nächsten Netzanschlusspunkt: ", style={
                                      "color": "grey"}), distance, "m", html.Br(),
                            html.Span("Wertung: ", style={
                                      "color": "grey"}), distance_score, html.Br(),
                            html.Span("Rang: ", style={"color": "grey"}), "--"
                        ]),
                    ], class_name="h-100")
                ], width={"size": 4}),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H4("Sonnenpotential", className="card-title"),
                            html.Span("Durchschnittliche Wh/m2 pro Monat: ",
                                      style={"color": "grey"}), irradiation, html.Br(),
                            html.Span("Wertung: ", style={
                                      "color": "grey"}), irradiation_score, html.Br(),
                            html.Span("Rang: ", style={"color": "grey"}), "--"
                        ]),
                    ], class_name="h-100")
                ], width={"size": 4})
            ], class_name="m-1")
        ]

    else:
        return dbc.Row()

###
# Run app
###

if __name__ == '__main__':
        app.run_server(mode="inline", debug = True, height = 1000)

In [1]:
# app backend
app = JupyterDash(prevent_initial_callbacks=True)

# Create geojsons.
kreis_geojson = dl.GeoJSON(url=f"/assets/kreis.json",  # url to geojson file
                     zoomToBounds=True,  # when true, zooms to bounds when data changes (e.g. on load)
                     zoomToBoundsOnClick=True,  # when true, zooms to bounds of feature (e.g. polygon) on click
                     hoverStyle=arrow_function(dict(weight=5, color='#666', dashArray='')),  # style applied on hover
                     #hideout=dict(colorscale=colorscale, classes=classes, style=style, colorProp="Score"),
                     id="geojson")

app.layout = html.Div([dl.Map(children=[dl.TileLayer(), kreis_geojson]),
                       html.Div(id="hidden-div", style={"display":"none"})],
                      style={'width': '50%', 'height': '75vh', 'margin': "auto", "display": "block"}, id="map")

@app.callback([Output("hidden-div", "children")], [Input("map", 'zoom')])
def func(viewport):
    print(viewport)
    return [html.A("Test")]

if __name__ == '__main__':
        app.run_server(mode="inline")

NameError: name 'JupyterDash' is not defined

In [2]:
import dash
from jupyter_dash import JupyterDash
import dash_leaflet as dl
import dash_html_components as html
import json

from dash.dependencies import Output, Input, State

app = JupyterDash(prevent_initial_callbacks=True)
app.layout = html.Div([
dl.Map(dl.TileLayer(), style={'width': '1000px', 'height': '500px'}, id="map"),
html.Div(id="output1"), html.Div(id="output2"), html.Div(id="output3")
])

@app.callback(Output("output1", "children"), [Input("map", "zoom")])
def func(viewport):
        print(viewport)
        return json.dumps(viewport)

@app.callback(Output("output2", "children"), [Input("map", "center")])
def func(viewport):
        print(viewport)
        return json.dumps(viewport)

@app.callback(Output("output3", "children"), [Input("map", "viewport")])
def func(viewport):
        print(viewport)
        return json.dumps(viewport)

if __name__ == '__main__':
        app.run_server(mode="inline")

In [3]:
# run app
if __name__ == '__main__':
    app.run_server(mode="inline")